In [ ]:
import qutip as qt
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import time

import pandas as pd 

from scipy import optimize
from scipy.optimize import minimize
from scipy.integrate import solve_ivp
from scipy.linalg import sqrtm

from mpl_toolkits.axes_grid1.inset_locator import inset_axes

#### parallelization 
options = qt.Options(num_cpus=6)



In [ ]:

### Setup matplotlib

plt.rcParams.update({
    "text.usetex": True,
    "text.latex.preamble": r"\usepackage[T1]{fontenc} \usepackage{bbm} \usepackage{lmodern} \usepackage{amsmath} \usepackage{amstext}  \usepackage{amsfonts}",
    "font.family": "serif",
    "font.serif": "STIX",
    "mathtext.fontset": "stix",
    "font.size": 10,
    "font.weight": 'bold',
    
    "savefig.dpi" : 300,
    "xtick.minor.visible": True, 
    "ytick.minor.visible": True,
    
    "ytick.right": True,
    "ytick.left": True,

    "xtick.top": True,
    "xtick.bottom": True,
 
    #
    "xtick.direction": "in",
    "ytick.direction": "in",
    #
    "xtick.major.width": 1.5,     # major tick width in points
    "ytick.major.width": 1.5,     # major tick width in points
    #
    "xtick.minor.width": 1.5,     # minor tick width in points
    "ytick.minor.width": 1.5,     # minor tick width in points
    #
    "xtick.major.pad": 3.0,     # distance to major tick label in points
    "ytick.major.pad": 3.0,     # distance to major tick label in points
    #
    "xtick.minor.pad": 1.5,     # distance to the minor tick label in points
    "ytick.minor.pad": 1.5,     # distance to the minor tick label in points
    #
    "xtick.major.size": 3.0, 
    "ytick.major.size": 3.0,
    
    "xtick.minor.size": 3.0, 
    "ytick.minor.size": 3.0,
    #
    "xtick.labelsize": 20, 
    "ytick.labelsize": 20,
    #
    "legend.frameon": True
})

##############
####### 

In [ ]:


def Get_maximum(vector):
    """
    This function returns the maximum value and the corresponding index in a given vector.
    
    Parameters:
    vector: array-like 
        Input vector to search for the maximum value. 
    
    Returns: 
    max_value: float
        Maximum value found in the input vector.
    aux_idx: int
        Index of the maximum value in the input vector.
    """
    
    # Initialize variables for index and maximum value
    aux_idx = 0
    max_value = -np.Infinity
    
    # Enumerate through the vector, checking each value
    for idx, value in enumerate(vector):
        
        # If the current maximum value is greater than the real part of the current value
        if np.real(max_value) < np.real(value): 
            
            # Update the maximum value and the corresponding index
            max_value = value
            aux_idx = idx
            
    # Return the maximum value and its index
    return max_value, aux_idx



def Numeric_diagonalization(Ws, side="left"):
    """
    This function performs the numeric diagonalization of a given matrix, 
    returning the maximum eigenvalue and the corresponding eigenvector.

    Parameters:
    Ws: array-like
        Matrix to be diagonalized. 

    Returns:
    e_values[idx]: float
        Dominant (maximum) eigenvalue of the matrix.
    e_rightvectors[idx]: array-like
        Eigenvector corresponding to the dominant eigenvalue.
    """
    # Calculate the eigenvalues and left eigenvectors of the transpose of the input matrix
    if side == "left":
        e_values, e_vectors = np.linalg.eig(np.conjugate(Ws).T)
    elif side == "right":
        e_values, e_vectors = np.linalg.eig(Ws)
    else: raise ValueError('You have to choose or left or right')
         
    # Get the maximum eigenvalue and its corresponding index
    dominant_e, idx = Get_maximum(e_values)

    # Get dimension
    d = int(np.sqrt(np.shape(Ws)[0]))
    print("Dimension: {} \n".format(d))
    
    # reshaping the result
    e_mat = np.reshape(e_vectors[:,idx], (d,d))
        
    # Return the dominant eigenvalue and corresponding (left or right) eigen matrix
    return e_values[idx], e_mat/np.trace(e_mat)


In [ ]:
def entropy_prod_current(t, jump, args):
    ### if hot bath

    if jump == 0:  # |0><1| 
        return -args["beta_h"](t, args)*(args["omega"] - args["Omega"])
    elif jump == 1:# |1><0|
        return args["beta_h"](t, args)*(args["omega"] - args["Omega"])
    elif jump == 2:# |0><2|
        return -args["beta_h"](t, args)*(args["omega"] + args["Omega"])
    elif jump == 3:# |2><0|
        return args["beta_h"](t, args)*(args["omega"] + args["Omega"])
    elif jump == 4:# |1><3|
        return -args["beta_h"](t, args)*(args["omega"] + args["Omega"])
    elif jump == 5:# |3><1|
        return args["beta_h"](t, args)*(args["omega"] + args["Omega"])
    elif jump == 6:# |2><3|
        return -args["beta_h"](t, args)*(args["omega"] - args["Omega"])
    elif jump == 7:# |3><2|
        return args["beta_h"](t, args)*(args["omega"] - args["Omega"])
    
    ### if cold bath

    elif jump == 8:# |0><1|
        return -args["beta0_c"]*(args["omega"] - args["Omega"])
    elif jump == 9:# |1><0|
        return args["beta0_c"]*(args["omega"]  - args["Omega"])
    elif jump == 10:# |0><2|
        return -args["beta0_c"]*(args["omega"] + args["Omega"])
    elif jump == 11:# |2><0|
        return args["beta0_c"]*(args["omega"] + args["Omega"])
    elif jump == 12:# |1><3|
        return -args["beta0_c"]*(args["omega"] + args["Omega"])
    elif jump == 13:# |3><1|
        return args["beta0_c"]*(args["omega"] + args["Omega"])
    elif jump == 14:# |2><3|
        return -args["beta0_c"]*(args["omega"] - args["Omega"])
    elif jump == 15:# |3><2|
        return args["beta0_c"]*(args["omega"] - args["Omega"])
    
    
    
def organize_jumps(col_times, col_which, Nboxes, args):
    # Set dt
    dt = args["tau"]/(Nboxes)
    # Initialize N empty arrays
    times = [[] for _ in range(Nboxes)]
    jumps = [[] for _ in range(Nboxes)]
    entropy_current = [0 for _ in range(Nboxes)]
    
    # Organize time instants into arrays
    for i, t in enumerate(col_times):
        n = int(t // dt)
        times[n].append(t)
        jumps[n].append(col_which[i])
        entropy_current[n] += entropy_prod_current(t, col_which[i], args)
    return times, jumps, entropy_current

In [ ]:
def Tilted_lindbladian(t, H0, cops, args):

    
    omega = args["omega"]
    Omega = args["Omega"]
    s = args["s_func"](t, args)
    bh = args["beta_h"](t, args)
    bc = args["beta0_c"]
    
    
    Ide = qt.tensor(qt.qeye(2), qt.qeye(2))

    Flat_H0  = -1.0j*(qt.tensor(Ide, H0) - qt.tensor(H0.trans(), Ide)).full()
     
    J_01_h, J_10_h, J_02_h, J_20_h, J_13_h, J_31_h, J_23_h, J_32_h = cops[0:8]
    J_01_c, J_10_c, J_02_c, J_20_c, J_13_c, J_31_c, J_23_c, J_32_c = cops[8:]
    
    ########### Flatenning the operators 
    Ide = qt.tensor(qt.qeye(2), qt.qeye(2))

    ####### Non-hermitian evolution 
    ## hot bath
    Flat_J_01_h = -(1.0/2.0)*(qt.tensor(Ide, J_01_h.dag()*J_01_h) + qt.tensor((J_01_h.dag()*J_01_h).trans(), Ide)).full()
    Flat_J_02_h = -(1.0/2.0)*(qt.tensor(Ide, J_02_h.dag()*J_02_h) + qt.tensor((J_02_h.dag()*J_02_h).trans(), Ide)).full()
    Flat_J_13_h = -(1.0/2.0)*(qt.tensor(Ide, J_13_h.dag()*J_13_h) + qt.tensor((J_13_h.dag()*J_13_h).trans(), Ide)).full()
    Flat_J_23_h = -(1.0/2.0)*(qt.tensor(Ide, J_23_h.dag()*J_23_h) + qt.tensor((J_23_h.dag()*J_23_h).trans(), Ide)).full()

    Flat_J_10_h = -(1.0/2.0)*(qt.tensor(Ide, J_10_h.dag()*J_10_h) + qt.tensor((J_10_h.dag()*J_10_h).trans(), Ide)).full()
    Flat_J_20_h = -(1.0/2.0)*(qt.tensor(Ide, J_20_h.dag()*J_20_h) + qt.tensor((J_20_h.dag()*J_20_h).trans(), Ide)).full()
    Flat_J_31_h = -(1.0/2.0)*(qt.tensor(Ide, J_31_h.dag()*J_31_h) + qt.tensor((J_31_h.dag()*J_31_h).trans(), Ide)).full()
    Flat_J_32_h = -(1.0/2.0)*(qt.tensor(Ide, J_32_h.dag()*J_32_h) + qt.tensor((J_32_h.dag()*J_32_h).trans(), Ide)).full()

    ## cold bath
    Flat_J_01_c = -(1.0/2.0)*(qt.tensor(Ide, J_01_c.dag()*J_01_c) + qt.tensor((J_01_c.dag()*J_01_c).trans(), Ide)).full()
    Flat_J_02_c = -(1.0/2.0)*(qt.tensor(Ide, J_02_c.dag()*J_02_c) + qt.tensor((J_02_c.dag()*J_02_c).trans(), Ide)).full()
    Flat_J_13_c = -(1.0/2.0)*(qt.tensor(Ide, J_13_c.dag()*J_13_c) + qt.tensor((J_13_c.dag()*J_13_c).trans(), Ide)).full()
    Flat_J_23_c = -(1.0/2.0)*(qt.tensor(Ide, J_23_c.dag()*J_23_c) + qt.tensor((J_23_c.dag()*J_23_c).trans(), Ide)).full()

    Flat_J_10_c = -(1.0/2.0)*(qt.tensor(Ide, J_10_c.dag()*J_10_c) + qt.tensor((J_10_c.dag()*J_10_c).trans(), Ide)).full()
    Flat_J_20_c = -(1.0/2.0)*(qt.tensor(Ide, J_20_c.dag()*J_20_c) + qt.tensor((J_20_c.dag()*J_20_c).trans(), Ide)).full()
    Flat_J_31_c = -(1.0/2.0)*(qt.tensor(Ide, J_31_c.dag()*J_31_c) + qt.tensor((J_31_c.dag()*J_31_c).trans(), Ide)).full()
    Flat_J_32_c = -(1.0/2.0)*(qt.tensor(Ide, J_32_c.dag()*J_32_c) + qt.tensor((J_32_c.dag()*J_32_c).trans(), Ide)).full()

    Flat_Inc_h = (Flat_J_01_h + Flat_J_02_h + Flat_J_13_h + Flat_J_23_h
                + Flat_J_10_h + Flat_J_20_h + Flat_J_31_h + Flat_J_32_h)

    Flat_Inc_c = (Flat_J_01_c + Flat_J_02_c + Flat_J_13_c + Flat_J_23_c
                + Flat_J_10_c + Flat_J_20_c + Flat_J_31_c + Flat_J_32_c)

    ####### Jumps 
    # hot bath
    Flat_jump_J_01_h = (qt.tensor(J_01_h.conj(),J_01_h)).full()
    Flat_jump_J_02_h = (qt.tensor(J_02_h.conj(),J_02_h)).full()
    Flat_jump_J_13_h = (qt.tensor(J_13_h.conj(),J_13_h)).full()
    Flat_jump_J_23_h = (qt.tensor(J_23_h.conj(),J_23_h)).full()

    Flat_jump_J_10_h = (qt.tensor(J_10_h.conj(),J_10_h)).full()
    Flat_jump_J_20_h = (qt.tensor(J_20_h.conj(),J_20_h)).full()
    Flat_jump_J_31_h = (qt.tensor(J_31_h.conj(),J_31_h)).full()
    Flat_jump_J_32_h = (qt.tensor(J_32_h.conj(),J_32_h)).full()
    
    #cold bath
    Flat_jump_J_01_c = (qt.tensor(J_01_c.conj(),J_01_c)).full()
    Flat_jump_J_02_c = (qt.tensor(J_02_c.conj(),J_02_c)).full()
    Flat_jump_J_13_c = (qt.tensor(J_13_c.conj(),J_13_c)).full()
    Flat_jump_J_23_c = (qt.tensor(J_23_c.conj(),J_23_c)).full()

    Flat_jump_J_10_c = (qt.tensor(J_10_c.conj(),J_10_c)).full()
    Flat_jump_J_20_c = (qt.tensor(J_20_c.conj(),J_20_c)).full()
    Flat_jump_J_31_c = (qt.tensor(J_31_c.conj(),J_31_c)).full()
    Flat_jump_J_32_c = (qt.tensor(J_32_c.conj(),J_32_c)).full()

    Flat_Jump_h = (  np.exp(+s*(omega-Omega)*bh)*Flat_jump_J_01_h 
                   + np.exp(+s*(omega+Omega)*bh)*Flat_jump_J_02_h 
                   + np.exp(+s*(omega+Omega)*bh)*Flat_jump_J_13_h 
                   + np.exp(+s*(omega-Omega)*bh)*Flat_jump_J_23_h

                   + np.exp(-s*(omega-Omega)*bh)*Flat_jump_J_10_h 
                   + np.exp(-s*(omega+Omega)*bh)*Flat_jump_J_20_h 
                   + np.exp(-s*(omega+Omega)*bh)*Flat_jump_J_31_h 
                   + np.exp(-s*(omega-Omega)*bh)*Flat_jump_J_32_h)

    Flat_Jump_c = (  np.exp(+s*(omega-Omega)*bc)*Flat_jump_J_01_c 
                   + np.exp(+s*(omega+Omega)*bc)*Flat_jump_J_02_c 
                   + np.exp(+s*(omega+Omega)*bc)*Flat_jump_J_13_c 
                   + np.exp(+s*(omega-Omega)*bc)*Flat_jump_J_23_c

                   + np.exp(-s*(omega-Omega)*bc)*Flat_jump_J_10_c 
                   + np.exp(-s*(omega+Omega)*bc)*Flat_jump_J_20_c 
                   + np.exp(-s*(omega+Omega)*bc)*Flat_jump_J_31_c 
                   + np.exp(-s*(omega-Omega)*bc)*Flat_jump_J_32_c)

    Lst = Flat_H0 + Flat_Inc_h + Flat_Jump_h + Flat_Inc_c + Flat_Jump_c     


    return Lst.astype(complex)




In [ ]:
########## Rates 

#####
##### hot bath
#####

def J_01_h_t(t, args): 
    return J_01*(np.sqrt(args["alpha_h"] * (args["omega"] - args["Omega"]) ** 3 
                    * (1 + 1 / (np.exp(args["beta_h"](t, args) * (args["omega"] - args["Omega"])) - 1))) ) 
    
def J_10_h_t(t, args):
    return J_10*(np.sqrt(args["alpha_h"] * (args["omega"] - args["Omega"]) ** 3 
                    * (0 + 1 / (np.exp(args["beta_h"](t, args) * (args["omega"] - args["Omega"])) - 1)))) 
    
def J_02_h_t(t, args):
    return J_02*(np.sqrt(args["alpha_h"] * (args["omega"] + args["Omega"]) ** 3 
                    * (1 + 1 / (np.exp(args["beta_h"](t, args) * (args["omega"] + args["Omega"])) - 1)))) 
    
def J_20_h_t(t, args):
    return J_20*(np.sqrt(args["alpha_h"] * (args["omega"] + args["Omega"]) ** 3 
                    * (0 + 1 / (np.exp(args["beta_h"](t, args) * (args["omega"] + args["Omega"])) - 1))))     
    
def J_13_h_t(t, args):
    return J_13*(np.sqrt(args["alpha_h"] * (args["omega"] + args["Omega"]) ** 3 
                    * (1 + 1 / (np.exp(args["beta_h"](t, args) * (args["omega"] + args["Omega"])) - 1))))
    
def J_31_h_t(t, args):
    return J_31*(np.sqrt(args["alpha_h"] * (args["omega"] + args["Omega"]) ** 3 
                    * (0 + 1 / (np.exp(args["beta_h"](t, args) * (args["omega"] + args["Omega"])) - 1)))) 
    
def J_23_h_t(t, args):
    return J_23*(np.sqrt(args["alpha_h"] * (args["omega"] - args["Omega"]) ** 3 
                    * (1 + 1 / (np.exp(args["beta_h"](t, args) * (args["omega"] - args["Omega"])) - 1))))
       
def J_32_h_t(t, args):
    return J_32*(np.sqrt(args["alpha_h"] * (args["omega"] - args["Omega"]) ** 3 
                    * (0 + 1 / (np.exp(args["beta_h"](t, args) * (args["omega"] - args["Omega"])) - 1))))
  
#####
##### cold bath
#####

def J_01_c_t(t, args): 
    return J_01*(np.sqrt(args["alpha_c"] * (args["omega"] - args["Omega"]) ** 3 
                    * (1 + 1 / (np.exp(args["beta0_c"] * (args["omega"] - args["Omega"])) - 1))) ) 

def J_10_c_t(t, args):
    return J_10*(np.sqrt(args["alpha_c"] * (args["omega"] - args["Omega"]) ** 3 
                    * (0 + 1 / (np.exp(args["beta0_c"] * (args["omega"] - args["Omega"])) - 1)))) 
    
def J_02_c_t(t, args):
    return J_02*(np.sqrt(args["alpha_c"] * (args["omega"] + args["Omega"]) ** 3 
                    * (1 + 1 / (np.exp(args["beta0_c"] * (args["omega"] + args["Omega"])) - 1)))) 
    
def J_20_c_t(t, args):
    return J_20*(np.sqrt(args["alpha_c"] * (args["omega"] + args["Omega"]) ** 3 
                    * (0 + 1 / (np.exp(args["beta0_c"] * (args["omega"] + args["Omega"])) - 1))))     
    
def J_13_c_t(t, args):
    return J_13*(np.sqrt(args["alpha_c"] * (args["omega"] + args["Omega"]) ** 3 
                    * (1 + 1 / (np.exp(args["beta0_c"] * (args["omega"] + args["Omega"])) - 1))))
    
def J_31_c_t(t, args):
    return J_31*(np.sqrt(args["alpha_c"] * (args["omega"] + args["Omega"]) ** 3 
                    * (0 + 1 / (np.exp(args["beta0_c"] * (args["omega"] + args["Omega"])) - 1)))) 
    
def J_23_c_t(t, args):
    return J_23*(np.sqrt(args["alpha_c"] * (args["omega"] - args["Omega"]) ** 3 
                    * (1 + 1 / (np.exp(args["beta0_c"] * (args["omega"] - args["Omega"])) - 1))))
       
def J_32_c_t(t, args):
    return J_32*(np.sqrt(args["alpha_c"] * (args["omega"] - args["Omega"]) ** 3 
                    * (0 + 1 / (np.exp(args["beta0_c"] * (args["omega"] - args["Omega"])) - 1))))
    


In [ ]:
def get_List_Doob_system(H0, cops_func, dt, args):
    
    ####
    #### Get a list of the auxiliary system in each time for the trajectories  
    ####

    tilted_H = []
    tilted_cops = []
    tau = args["tau"]
    tlist = np.arange(0, tau, dt/2)
    
    Nsteps = int(tau/(tlist[1]-tlist[0]))
    
    data = {}
    for i, t in enumerate(tlist): 
    
        t0 = time.time()
        
        cops = [c(t, args) for c in cops_func]
        
        #### Tilted Lindbladian 
        Ls = Tilted_lindbladian(t, H0, cops, args)
        
        lambda0, l0 = Numeric_diagonalization(Ls, side="left")
        
        #### Non-hermitian term of the deterministic evolution
        
        Heff = H0
        for c in cops:
            Heff -= 0.5j * c.dag() * c
        ####
        
        l0sqr = qt.Qobj(l0, dims=[[2, 2], [2, 2]]).sqrtm()
        
        Heff = l0sqr * Heff * l0sqr.inv()
        
        cops_aux = [l0sqr*c*l0sqr.inv() for c in cops]

        data.update({i:[Heff, cops_aux]})
        
        if i % int(500) == 0:
            print("Doob rotation time: {}   -   Rotation {} of {} \n".format(time.time()-t0, i, Nsteps))
        
    data.update({"time":tlist})
    return data 
    

In [ ]:
def OneTrajectory(psi, Hdata):

    tlist = Hdata["time"]
    Nsteps = len(tlist)
    
    dt = tlist[2] - tlist[0]
    
    R = np.random.rand() 
    
    jumps = []
    times = []
    for i, t in enumerate(tlist): 
        
        if i % 2 == 0 and i < Nsteps-1: 
            
            Hauxt = Hdata[i][0]
            Hauxt2 = Hdata[i+1][0]
        
            k1 = -1.0j*Hauxt  * psi
            k2 = -1.0j*Hauxt2 * (psi + k1 * dt / 2)
            k3 = -1.0j*Hauxt2 * (psi + k2 * dt / 2)
            k4 = -1.0j*Hauxt  * (psi + k3 * dt)
    
            psi = psi + (dt / 6) * (k1 + 2 * k2 + 2 * k3 + k4)
        
            if psi.norm() < R:
                #print("We got a jump at time {} \n\n\n".format(t))

                prob = [(c * psi).norm()**2 for c in Hdata[i][1]] 
                prob = np.array(prob)/np.sum(prob)

                jump = np.random.choice(16, p=prob)
                
                #print("Jump: The norm {} and the random number {} \n".format(psi.norm(), R))

                psi = Hdata[i][1][jump] * psi                  
                #print("Norm after jump: {}".format(psi.norm()))
                psi = psi/psi.norm()
                #print("Norm after jump normalized: {}".format(psi.norm()))

                #print("The state", psi, "\n")

                jumps.append(jump)
                times.append(t)
                R = np.random.rand() 
                
            else:continue 
        else: continue 
            
    return jumps, times

In [ ]:
def get_trajectories(psi, Hdata, ntraj):
    jumps_list, times_list = [], []
    
    for traj in range(ntraj):
        t0 = time.time()
        jumps, times = OneTrajectory(psi, Hlist)
        jumps_list.append(jumps)
        times_list.append(times)
        if traj % 10 == 0: 
            print("Execution time: {}   -   Run {} of {}".format(time.time()-t0, traj, ntraj))
    return jumps_list, times_list

In [ ]:
# Define the states
ket00 = qt.tensor(qt.basis(2,1), qt.basis(2,1))
ket01 = qt.tensor(qt.basis(2,1), qt.basis(2,0))
ket10 = qt.tensor(qt.basis(2,0), qt.basis(2,1))
ket11 = qt.tensor(qt.basis(2,0), qt.basis(2,0))

sigmam_h = qt.tensor(qt.sigmam(), qt.qeye(2))
sigmam_c = qt.tensor(qt.qeye(2), qt.sigmam())
sigmap_h = qt.tensor(qt.sigmap(), qt.qeye(2))
sigmap_c = qt.tensor(qt.qeye(2), qt.sigmap())

sigmax_h = qt.tensor(qt.sigmax(), qt.qeye(2))
sigmax_c = qt.tensor(qt.qeye(2), qt.sigmax())

sigmaz_h = qt.tensor(qt.sigmaz(), qt.qeye(2))
sigmaz_c = qt.tensor(qt.qeye(2), qt.sigmaz())

eps0 = ket00
eps1 = np.sqrt(1/2)*(ket01 - ket10)
eps2 = np.sqrt(1/2)*(ket01 + ket10)
eps3 = ket11
    
##### Transitions 
    
J_01 = eps0 * eps1.dag()
    
J_10 = eps1 * eps0.dag() 
        
J_02 = eps0 * eps2.dag()
    
J_20 = eps2 * eps0.dag()
    
J_13 = eps1 * eps3.dag()

J_31 = eps3 * eps1.dag()

J_23 = eps2 * eps3.dag()
    
J_32 = eps3 * eps2.dag()

In [ ]:
####### Parameters 

omega = 1.0
alpha_h = 1.0
alpha_c = 1.0
beta0_h = 1.0/(2.0) # T = 2
beta0_c = 1.0/(1.0) # T = 1
Omega = 0.5
g = 1.0

def s_func(t, args):
    return np.tanh(2*t/args["tau"] - 1)
    #return 0 

def beta_h(t, args):
    return 1/(1/args["beta0_h"]   + np.sin(t*np.pi/args["tau"])**2)


############# Total dynamics time


tau = 100
#Nsteps = 1000


############# Argumets dict 

args  = {"omega":omega,
         "Omega":Omega,
         "g":g,
         "alpha_h":alpha_h,
         "alpha_c":alpha_c,
         "beta_h":beta_h,
         "beta0_h":beta0_h,
         "beta0_c":beta0_c,
         "s_func": s_func,
         "tau":tau}

In [ ]:
####### Hamiltonian    
    
H0 = (Omega*( sigmam_h*sigmap_c + sigmap_h*sigmam_c) 
        + g*(sigmax_c + sigmax_h) 
        + (omega/2)*(sigmaz_c + sigmaz_h))

#H0 = (omega/2)*(sigmaz_c + sigmaz_h)

cops_func = [J_01_h_t, J_10_h_t, 
             J_02_h_t, J_20_h_t,
             J_13_h_t, J_31_h_t, 
             J_23_h_t, J_32_h_t,
             J_01_c_t, J_10_c_t, 
             J_02_c_t, J_20_c_t,
             J_13_c_t, J_31_c_t, 
             J_23_c_t, J_32_c_t,]


####### Jump operators 

In [ ]:
#### Get the list of the auxliariy system at each time

dt = 0.01

Hlist = get_List_Doob_system(H0, cops_func, dt=dt, args=args)

print("We have {} intervals \n".format(int(tau/dt)))

In [ ]:
ntraj = 100

psi0 = qt.tensor(qt.basis(2,1), qt.basis(2,0))
jumps, times = get_trajectories(psi0, Hlist, ntraj)

In [ ]:
#### Script for evaluating the entropy production on trajectories


Nboxes = 10

entropy_data = []
for traj in range(ntraj):
    t, j, entropy = organize_jumps(times[traj], jumps[traj], Nboxes, args)
    entropy_data.append(entropy)
    
entropy_data = np.array(entropy_data)

mean_values = np.mean(np.array(entropy_data).T, axis=1)
std_dev_values = np.std(np.array(entropy_data).T, axis=1)

In [ ]:
plt.scatter(np.arange(Nboxes)/Nboxes, mean_values)

In [ ]:
b = qt.Qobj(np.random.rand(3,3))

In [ ]:
b.sqrtm()

In [ ]:
b